# Final Project Notebook
Group: 9
Group Members: Shawn Ericksen (ericksen@uark.edu), Garret Fulghum (gmfulghu@uark.edu), Wesley Parker (wgparker@uark.edu)

This practice project focuses on the Microsoft Malware Prediction dataset. This can be accessed from: https://www.kaggle.com/competitions/microsoft-malware-prediction/data

Below is all of our specified imports

In [20]:
import numpy as np
import pandas as pd
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
from dask import dataframe as dd
import dask.array as da
from dask_ml.preprocessing import OneHotEncoder, LabelEncoder, DummyEncoder, Categorizer
from dask.distributed import Client

from sklearn.naive_bayes import CategoricalNB

In [2]:
client = Client(n_workers=2)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51179,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:51191,Total threads: 4
Dashboard: http://127.0.0.1:51192/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:51183,


In [23]:
def clean(df):
    drop_cols = []
    for col in df.columns:
#         if df[col].dtype == 'float16' or df[col].dtype == 'float32' or df[col].dtype == 'float64':
#             df[col] = (df[col] >= df.mean(numeric_only=True)[col]).astype(int)
        if df[col].dtype == 'float16' or df[col].dtype == 'float32' or df[col].dtype == 'float64':
            drop_cols.append(col)
            
    df.drop(labels=cols, axis=1)
    
    return df

In [4]:
def encode(train_data):
    df = clean(train_data)

    X_train = df.drop(columns=['HasDetections'])
    encoder = DummyEncoder()
    X_train = encoder.fit_transform(X_train)
    y_train = df['HasDetections'].values
    y_enc = LabelEncoder().fit(y_train)
    y_train = y_enc.transform(y_train)
    
#     df = clean(test_data, include_continuous)
    
#     X_test = df.drop(columns=['income'])
#     X_test = encoder.transform(X_test).toarray()
#     y_test = y_enc.transform(df['income'].values)
    
    return X_train, y_train, y_enc

In [5]:
dtypes={'MachineIdentifier': 'string', 'ProductName': 'category', 'EngineVersion': 'category', 'AppVersion': 'category', 'AvSigVersion': 'category', 'IsBeta': 'float16', 'RtpStateBitfield': 'category', 'IsSxsPassiveMode': 'float16', 'DefaultBrowsersIdentifier': 'float32', 'AVProductStatesIdentifier': 'float32', 'AVProductsInstalled': 'float16', 'AVProductsEnabled': 'float16', 'HasTpm': 'float16', 'CountryIdentifier': 'category', 'CityIdentifier': 'float32', 'OrganizationIdentifier': 'float32', 'GeoNameIdentifier': 'category', 'LocaleEnglishNameIdentifier': 'category', 'Platform': 'category', 'Processor': 'category', 'OsVer': 'category', 'OsBuild': 'category', 'OsSuite': 'category', 'OsPlatformSubRelease': 'category', 'OsBuildLab': 'category', 'SkuEdition': 'category', 'IsProtected': 'category', 'AutoSampleOptIn': 'category', 'PuaMode': 'category', 'SMode': 'float16', 'IeVerIdentifier': 'float32', 'SmartScreen': 'category', 'Firewall': 'float16', 'UacLuaenable': 'float32', 'Census_MDC2FormFactor': 'category', 'Census_DeviceFamily': 'category', 'Census_OEMNameIdentifier': 'category', 'Census_OEMModelIdentifier': 'float32', 'Census_ProcessorCoreCount': 'float16', 'Census_ProcessorManufacturerIdentifier': 'category', 'Census_ProcessorModelIdentifier': 'category', 'Census_ProcessorClass': 'category', 'Census_PrimaryDiskTotalCapacity': 'float64', 'Census_PrimaryDiskTypeName': 'category', 'Census_SystemVolumeTotalCapacity': 'float32', 'Census_HasOpticalDiskDrive': 'float16', 'Census_TotalPhysicalRAM': 'float32', 'Census_ChassisTypeName': 'category', 'Census_InternalPrimaryDiagonalDisplaySizeInInches': 'float32', 'Census_InternalPrimaryDisplayResolutionHorizontal': 'float32', 'Census_InternalPrimaryDisplayResolutionVertical': 'float32', 'Census_PowerPlatformRoleName': 'category', 'Census_InternalBatteryType': 'category', 'Census_InternalBatteryNumberOfCharges': 'float32', 'Census_OSVersion': 'category', 'Census_OSArchitecture': 'category', 'Census_OSBranch': 'category', 'Census_OSBuildNumber': 'category', 'Census_OSBuildRevision': 'category', 'Census_OSEdition': 'category', 'Census_OSSkuName': 'category', 'Census_OSInstallTypeName': 'category', 'Census_OSInstallLanguageIdentifier': 'float16', 'Census_OSUILocaleIdentifier': 'float16', 'Census_OSWUAutoUpdateOptionsName': 'category', 'Census_IsPortableOperatingSystem': 'float16', 'Census_GenuineStateName': 'category', 'Census_ActivationChannel': 'category', 'Census_IsFlightingInternal': 'float16', 'Census_IsFlightsDisabled': 'float16', 'Census_FlightRing': 'category', 'Census_ThresholdOptIn': 'float16', 'Census_FirmwareManufacturerIdentifier': 'category', 'Census_FirmwareVersionIdentifier': 'float32', 'Census_IsSecureBootEnabled': 'float16', 'Census_IsWIMBootEnabled': 'float16', 'Census_IsVirtualDevice': 'float16', 'Census_IsTouchEnabled': 'float16', 'Census_IsPenCapable': 'float16', 'Census_IsAlwaysOnAlwaysConnectedCapable': 'float16', 'Wdft_IsGamer': 'float16', 'Wdft_RegionIdentifier': 'float16', 'HasDetections': 'float16'}

In [30]:
filepath = "train.csv"

cols = list(pd.read_csv(filepath, nrows=1))[1:]

# chunk = dd.read_csv("train.csv", dtype=dtypes, na_values=['NA', 'UNKNOWN', 'NOT_SET', 'nan'], usecols =[i for i in cols if i != 'MachineIdentifier'], chunksize=100000)

# df_train = pd.concat(chunk)

df_train = dd.read_csv(filepath, dtype=dtypes, blocksize="32MB", na_values=['NA', 'UNKNOWN', 'NOT_SET', 'nan'], usecols =[i for i in cols if i != 'MachineIdentifier']).head(n=50000)
# df_train.ProductName.cat.known
# df_train = df_train.categorize()

# for column in df_train.columns:
#     print(pd.api.types.infer_dtype(df[column]))
#     df_train[column] = df_train[column].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))
#     df_train[column][df_train[column].apply(lambda x: isinstance(x, type))]

In [ ]:
df_train = df_train.categorize()

In [31]:
df_train.memory_usage(deep=True, index=False).sort_values(ascending=False)

Census_PrimaryDiskTotalCapacity     400000
AvSigVersion                        332281
Census_FirmwareVersionIdentifier    200000
Census_TotalPhysicalRAM             200000
Census_OEMModelIdentifier           200000
                                     ...  
Census_DeviceFamily                  50143
ProductName                          50129
IsProtected                          50116
PuaMode                              50059
AutoSampleOptIn                      50058
Length: 82, dtype: int64

In [32]:
df_train.memory_usage(deep=True, index=False).sum()

8761575

In [ ]:
df_train.ProductName.cat.known

In [ ]:
coo = sparse.coo_matrix(df_train)
csr = coo.tocsr()
print(csr)

In [ ]:
# df_train.to_parquet("train.parquet", compression="snappy", engine="fastparquet")

In [ ]:
print("- - Adult.Data - -")
print("Number of lines present: ", len(df_train))
print("Number of Columns: ", len(df_train.columns))

In [ ]:
topCount = 10
print("Top ", topCount, " dataFrames:")
print(df_train.head(topCount))

In [ ]:
for x in range(0, len(df_train.dtypes)):
    print(str(df_train.columns[x]) + ": " + str(df_train.dtypes[x]) + ": " + str(df_train.to_numpy()[1][x]))

# s = "{"
# for x in range(0, len(df_train.dtypes)):
#     if str(df_train.dtypes[x]) == "Int16":
#         s += "'" + str(df_train.columns[x]) + "': 'Int64"
#     elif str(df_train.dtypes[x]) == "float64":
#         s += "'" + str(df_train.columns[x]) + "': 'float16"
#     else:
#         s += "'" + str(df_train.columns[x]) + "': '" + str(df_train.dtypes[x])
#     if x != len(df_train.dtypes) - 1:
#         s += "', "
#     else:
#         s += "'}"

# print(s)

In [ ]:
# df_train = pd.read_csv("train.csv", header=0)

In [33]:
X_train, y_train, y_enc = encode(df_train)

In [34]:
X_train.shape

(50000, 7751)

In [35]:
clf1 = CategoricalNB().fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for 'int'.